In [ ]:
# Import required packages
from os import path 
import time
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from torch.optim import AdamW
from tqdm import tqdm, trange
import zuko
from zuko.flows import Distribution, NSF
from zuko.distributions import DiagNormal, BoxUniform, Minimum
from zuko.flows import DistributionModule, FlowModule, Unconditional
from hnne import HNNE

from utils.settings import config
from utils.utils import *
from utils.model import *
from utils.robot import Robot
from utils.dataset import create_dataset

In [ ]:
panda = Robot(verbose=False)
# data generation
X, y = load_data(robot=panda, num_samples=250_0000)
# build dimension reduction model
hnne, ds, loader = get_hnne_model(X, y)
# Build Generative model, NSF
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer = get_flow_model()

In [ ]:
df, err = test_l2_err(config, robot=panda, loader=loader, model=flow)
ax1 = df.plot.scatter(x='log_prob', y='l2_err')
df.describe()

In [ ]:
nflow = get_nflow_model(flow=flow)
df, err = test_l2_err(config, robot=panda, loader=loader, model=nflow)
ax1 = df.plot.scatter(x='log_prob', y='l2_err')
df.describe()

In [ ]:
if path.exists(path=config.ee_traj_path):
    ee_traj = load_numpy(file_path=config.ee_traj_path)
    q_traj = load_numpy(file_path=config.q_traj_path)
else:
    ee_traj, q_traj = panda.path_generate_via_stable_joint_traj()
    save_numpy(file_path=config.ee_traj_path, arr=ee_traj)
    save_numpy(file_path=config.q_traj_path, arr=q_traj)

In [ ]:
def sample_jtraj(path, pidx, model=flow):
    path_len = len(path)
    pidx = np.tile(pidx, (path_len,1))
    cstd = np.zeros((path_len,))
    
    y = np.column_stack((path, pidx, cstd))
    y = torch.tensor(data=y, device='cuda', dtype=torch.float32)
    
    errs = np.zeros((len(path),))
    log_probs = np.zeros((len(path),))
    
    step = 0
    x_hat = model(y).sample((1,))
    log_prob = model(y).log_prob(x_hat)
    
    x_hat = x_hat.detach().cpu().numpy()[0]
    log_prob = -log_prob.detach().cpu().numpy()[0]
    print(path.shape)
    print(x_hat.shape)
    print(log_prob.shape)
    for q, lp, ee_pos in zip(x_hat, log_prob, path):
        errs[step] = panda.dist_fk(q=q, ee_pos=ee_pos)
        log_probs[step] = lp     
        step += 1
    print(f'step={step}')
    df = pd.DataFrame(np.column_stack((errs, log_probs)), columns=['l2_err', 'log_prob'])
    qs = x_hat
    return df, qs

In [ ]:
pidx = hnne.transform(X=q_traj[0:3])
pidx

In [ ]:
for i, px in enumerate(pidx):
    df, qs = sample_jtraj(ee_traj, px, flow)
    print(df.describe())
    save_numpy(file_path=f'./data/exp_qs_{i}.npy', arr=qs)

In [ ]:
ee_traj = load_numpy(file_path=config.ee_traj_path)
err = np.zeros((100,))

for i in range(3):
    step = 0
    qs = load_numpy(file_path=f'./data/exp_qs_{i}.npy')
    for i in range(100):
        err[i] = panda.dist_fk(q=qs[i], ee_pos=ee_traj[i])
    outliner = np.where(err > 0.05)
    print(outliner)
    print(err[outliner])
    print(np.sum(err))
    # panda.plot_qs(qs)